<a href="https://colab.research.google.com/github/szh141/UVA-AMF/blob/main/lamda_body_intensity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import glob
import os
extension = '*.jpg'
file_list = glob.glob(extension)

overlay = []
ch0 = []
ch1 = []
file = []


for file_name in file_list:
  if 'ch00' in file_name:
    ch0.append(file_name)
  elif 'ch01' in file_name:
      ch1.append(file_name)
  elif 'overlay' in file_name:
        overlay.append(file_name)

overlay.sort()
ch0.sort()
ch1.sort()

for i in overlay:
  name = i.split('_')
  file.append(name[0])

In [2]:
pip install stardist

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 11.3 MB/s eta 0:00:00


In [3]:
from skimage import io, filters, morphology, feature,color
from matplotlib import pyplot as plt
import numpy as np

In [8]:
from stardist.models import StarDist2D
model = StarDist2D.from_pretrained('2D_versatile_fluo')

from stardist.data import test_image_nuclei_2d
from stardist.plot import render_label
from csbdeep.utils import normalize
import matplotlib.pyplot as plt

from skimage.measure import regionprops_table
from skimage.segmentation import watershed

import pandas as pd

count = []
ave_intensity = []

for i in range(len(overlay)):
  img_overlay = io.imread(overlay[i])
  #plt.imshow(img_overlay)
  #plt.show()

  #print(img_overlay.shape)
  img_gray = color.rgb2gray(img_overlay)
  #plt.imshow(img_gray,vmax=0.3)
  #plt.show()

  nuclear = io.imread(ch0[i])[...,2]
  print(nuclear.shape)
  #plt.imshow(nuclear,vmax=100)
  #plt.show()

  img_green = io.imread(ch1[i])[...,1]
  #plt.imshow(img_green,vmax=100)

  mask = img_gray > 0.02
  mask2 = morphology.remove_small_objects(mask, min_size=100)
  #plt.imshow(mask)
  #plt.show()
  #plt.imshow(mask2)



  img = nuclear.copy()
  img.dtype

  labels, _ = model.predict_instances(normalize(img))


  cell_seg = watershed(img_gray, labels, mask=mask2)
  props = regionprops_table(cell_seg, intensity_image=img_green,
                            properties=('label',
                                        'intensity_mean',
                                        'area',
                                                  ))

  df = pd.DataFrame(props)
  df['total_intensity'] = df['intensity_mean']*df['area']
  #df.head()
  #df.hist(column='total_intensity',range=[30000,80000])
  df_intensity = df[(df['total_intensity']>11000)&(df['area']>1300)]

  intensity = f'{file[i]}.csv'
  df_intensity.to_csv(intensity, index=False)

  ave_intensity.append(df_intensity['total_intensity'].mean())


  count.append(df_intensity.shape[0])



Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.
(1024, 1024)
(1024, 1024)
(1024, 1024)
(1024, 1024)
(1024, 1024)


In [9]:
df_summary = pd.DataFrame({'file_name': file, 'bright_cell_count': count,'average_intensity':ave_intensity})

df_summary.to_csv('summary.csv',index=False)